In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
import matplotlib as plt

#NLP
import spacy
import gensim
from spacy.tokenizer import Tokenizer
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS as SW
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

import pyLDAvis
import pyLDAvis.gensim_models

C:\Users\rishabhstha\AppData\Local\Programs\Python\Python38\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
news_data=pd.read_csv("newsData/JWCallNewsData.csv")
news_data

C:\Users\rishabhstha\AppData\Local\Programs\Python\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,Date,Media,Title,Article,Summary,Keyword
0,0,2-Mar-21,NBC News,"Jackson, Mississippi, water crisis brings to l...",After more than two weeks without clean runnin...,Residents have been lining up at several sites...,Jackson Mississippi water crisis
1,1,2-Mar-21,CNN,"In Jackson, Mississippi, some residents have b...","(CNN) Many residents of Jackson, Mississippi, ...","(CNN) Many residents of Jackson, Mississippi, ...",Jackson Mississippi water crisis
2,2,3-Mar-21,USA Today,"Weeks after Mississippi winter storms, some re...","Weeks after Mississippi winter storms, some re...","Weeks after Mississippi winter storms, some re...",Jackson Mississippi water crisis
3,3,15-Mar-21,Slate,Ignore Mississippi’s Water Crisis at Your Own ...,"For four weeks now, thousands of residents of ...","Water mains were damaged, reservoirs froze, an...",Jackson Mississippi water crisis
4,4,"th · Mar 22, 2021",The New York Times,‘You Can’t Bathe. You Can’t Wash.’ Water Crisi...,Mr. Reeves called in the National Guard last m...,Mr. Reeves called in the National Guard last m...,Jackson Mississippi water crisis
...,...,...,...,...,...,...,...
219,43,2021-03-04 06:25:16+00:00,Our Daily Planet,"Jackson, MS is Struggling With a Water Crisis,...","by Amy Lupica, ODP Staff Writer\n\nWhile all e...","by Amy Lupica, ODP Staff WriterWhile all eyes ...",NaN
220,44,2021-02-19 22:33:00+03:00,Daily Sabah,US cities slammed by storms face new crisis: W...,States slammed by winter storms that left mill...,States slammed by winter storms that left mill...,NaN
221,0,3-Mar-21,Jackson Free Press,"PUBLISHER'S NOTE: Jackson’s Water Crisis, What...",While Lt. Gov. Delbert Hosemann often presents...,Around a long rectangle of tables in the middl...,Nick Judin Jackson water crisis
222,5,2021-03-02 01:54:57+00:00,Mississippi Free Press,Hosemann Pins Water Blame on Jackson Leaders S...,Lt. Gov. Delbert Hosemann confirmed today that...,Delbert Hosemann confirmed today that there ar...,Nick Judin Jackson water crisis


In [3]:
#Preprocessing- Cleaning, Tokenizing

def clean_text(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text=re.sub('[^A-Za-z0-9]+', ' ', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

news_data['Clean Text']=news_data['Article'].apply(lambda x:clean_text(x))


def remove_stopwords(x):
    stopword=stopwords.words('english')
    stopword.append("amp")
    stopword.append("t")
    stopword.append("co")
    
   
    return [y for y in x if y not in stopword]


news_data['word list']=news_data['Clean Text'].apply(lambda x:str(x).split())
news_data['word list']=news_data['word list'].apply(lambda x:remove_stopwords(x))

C:\Users\rishabhstha\AppData\Local\Programs\Python\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:5: DeprecationWarning: invalid escape sequence \[
<>:6: DeprecationWarning: invalid escape sequence \S
<>:10: DeprecationWarning: invalid escape sequence \w
<>:5: DeprecationWarning: invalid escape sequence \[
<>:6: DeprecationWarning: invalid escape sequence \S
<>:10: DeprecationWarning: invalid escape sequence \w
<ipython-input-3-54ce614a7ca9>:5: DeprecationWarning: invalid escape sequence \[
  text = re.sub('\[.*?\]', '', text)
<ipython-input-3-54ce614a7ca9>:6: DeprecationWarning: invalid escape sequence \S
  text = re.sub('https?://\S+|www\.\S+', '', text)
<ipython-input-3-54ce614a7ca9>:10: De

In [4]:
news_data

C:\Users\rishabhstha\AppData\Local\Programs\Python\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,Date,Media,Title,Article,Summary,Keyword,Clean Text,word list
0,0,2-Mar-21,NBC News,"Jackson, Mississippi, water crisis brings to l...",After more than two weeks without clean runnin...,Residents have been lining up at several sites...,Jackson Mississippi water crisis,after more than two weeks without clean runnin...,"[two, weeks, without, clean, running, water, k..."
1,1,2-Mar-21,CNN,"In Jackson, Mississippi, some residents have b...","(CNN) Many residents of Jackson, Mississippi, ...","(CNN) Many residents of Jackson, Mississippi, ...",Jackson Mississippi water crisis,cnn many residents of jackson mississippi hav...,"[cnn, many, residents, jackson, mississippi, w..."
2,2,3-Mar-21,USA Today,"Weeks after Mississippi winter storms, some re...","Weeks after Mississippi winter storms, some re...","Weeks after Mississippi winter storms, some re...",Jackson Mississippi water crisis,weeks after mississippi winter storms some res...,"[weeks, mississippi, winter, storms, residents..."
3,3,15-Mar-21,Slate,Ignore Mississippi’s Water Crisis at Your Own ...,"For four weeks now, thousands of residents of ...","Water mains were damaged, reservoirs froze, an...",Jackson Mississippi water crisis,for four weeks now thousands of residents of j...,"[four, weeks, thousands, residents, jackson, m..."
4,4,"th · Mar 22, 2021",The New York Times,‘You Can’t Bathe. You Can’t Wash.’ Water Crisi...,Mr. Reeves called in the National Guard last m...,Mr. Reeves called in the National Guard last m...,Jackson Mississippi water crisis,mr reeves called in the national guard last mo...,"[mr, reeves, called, national, guard, last, mo..."
...,...,...,...,...,...,...,...,...,...
219,43,2021-03-04 06:25:16+00:00,Our Daily Planet,"Jackson, MS is Struggling With a Water Crisis,...","by Amy Lupica, ODP Staff Writer\n\nWhile all e...","by Amy Lupica, ODP Staff WriterWhile all eyes ...",NaN,by amy lupica odp staff writer while all eyes ...,"[amy, lupica, odp, staff, writer, eyes, texas,..."
220,44,2021-02-19 22:33:00+03:00,Daily Sabah,US cities slammed by storms face new crisis: W...,States slammed by winter storms that left mill...,States slammed by winter storms that left mill...,NaN,states slammed by winter storms that left mill...,"[states, slammed, winter, storms, left, millio..."
221,0,3-Mar-21,Jackson Free Press,"PUBLISHER'S NOTE: Jackson’s Water Crisis, What...",While Lt. Gov. Delbert Hosemann often presents...,Around a long rectangle of tables in the middl...,Nick Judin Jackson water crisis,while lt gov delbert hosemann often presents h...,"[lt, gov, delbert, hosemann, often, presents, ..."
222,5,2021-03-02 01:54:57+00:00,Mississippi Free Press,Hosemann Pins Water Blame on Jackson Leaders S...,Lt. Gov. Delbert Hosemann confirmed today that...,Delbert Hosemann confirmed today that there ar...,Nick Judin Jackson water crisis,lt gov delbert hosemann confirmed today that t...,"[lt, gov, delbert, hosemann, confirmed, today,..."


In [5]:
news_data['word list'][0]

C:\Users\rishabhstha\AppData\Local\Programs\Python\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['two',
 'weeks',
 'without',
 'clean',
 'running',
 'water',
 'katasha',
 'johnson',
 'saw',
 'slow',
 'trickle',
 'dribbling',
 'faucet',
 'home',
 'west',
 'side',
 'jackson',
 'mississippi',
 'tuesday',
 'brown',
 'drip',
 'feel',
 'like',
 'reprieve',
 'johnson',
 'lost',
 'water',
 'back',
 'back',
 'winter',
 'storms',
 'slammed',
 'city',
 'century',
 'old',
 'water',
 'infrastructure',
 'last',
 'month',
 'enough',
 'anything',
 'even',
 'look',
 'safe',
 'said',
 'johnson',
 'mother',
 'children',
 'ages',
 'instead',
 'johnson',
 'fianc',
 'using',
 'melted',
 'snow',
 'collected',
 'rainwater',
 'stored',
 'four',
 'large',
 'coolers',
 'flush',
 'toilet',
 'clean',
 'dishes',
 'hands',
 'johnson',
 'boils',
 'pots',
 'water',
 'taken',
 'bathtub',
 'pre',
 'filled',
 'water',
 'available',
 'absolutely',
 'horrible',
 'live',
 'like',
 'said',
 'jackson',
 'entered',
 'third',
 'week',
 'crisis',
 'left',
 'much',
 'city',
 'without',
 'water',
 'since',
 'freezing',
 'tem

In [7]:
#Removing stopwords, repeated words 
def remove_stopwords(x):
    stopword=stopwords.words('english')
    stopword.append('water')
    stopword.append('jackson')
    stopword.append('mississippi')
    stopword.append('ms')
    stopword.append('city')
    stopword.append('said')

    return [y for y in x if y not in stopword]

news_data['No stopword list']=news_data['word list'].apply(lambda x:remove_stopwords(x))
news_data

C:\Users\rishabhstha\AppData\Local\Programs\Python\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,Date,Media,Title,Article,Summary,Keyword,Clean Text,word list,No stopword list
0,0,2-Mar-21,NBC News,"Jackson, Mississippi, water crisis brings to l...",After more than two weeks without clean runnin...,Residents have been lining up at several sites...,Jackson Mississippi water crisis,after more than two weeks without clean runnin...,"[two, weeks, without, clean, running, water, k...","[two, weeks, without, clean, running, katasha,..."
1,1,2-Mar-21,CNN,"In Jackson, Mississippi, some residents have b...","(CNN) Many residents of Jackson, Mississippi, ...","(CNN) Many residents of Jackson, Mississippi, ...",Jackson Mississippi water crisis,cnn many residents of jackson mississippi hav...,"[cnn, many, residents, jackson, mississippi, w...","[cnn, many, residents, without, since, winter,..."
2,2,3-Mar-21,USA Today,"Weeks after Mississippi winter storms, some re...","Weeks after Mississippi winter storms, some re...","Weeks after Mississippi winter storms, some re...",Jackson Mississippi water crisis,weeks after mississippi winter storms some res...,"[weeks, mississippi, winter, storms, residents...","[weeks, winter, storms, residents, still, show..."
3,3,15-Mar-21,Slate,Ignore Mississippi’s Water Crisis at Your Own ...,"For four weeks now, thousands of residents of ...","Water mains were damaged, reservoirs froze, an...",Jackson Mississippi water crisis,for four weeks now thousands of residents of j...,"[four, weeks, thousands, residents, jackson, m...","[four, weeks, thousands, residents, potable, e..."
4,4,"th · Mar 22, 2021",The New York Times,‘You Can’t Bathe. You Can’t Wash.’ Water Crisi...,Mr. Reeves called in the National Guard last m...,Mr. Reeves called in the National Guard last m...,Jackson Mississippi water crisis,mr reeves called in the national guard last mo...,"[mr, reeves, called, national, guard, last, mo...","[mr, reeves, called, national, guard, last, mo..."
...,...,...,...,...,...,...,...,...,...,...
219,43,2021-03-04 06:25:16+00:00,Our Daily Planet,"Jackson, MS is Struggling With a Water Crisis,...","by Amy Lupica, ODP Staff Writer\n\nWhile all e...","by Amy Lupica, ODP Staff WriterWhile all eyes ...",NaN,by amy lupica odp staff writer while all eyes ...,"[amy, lupica, odp, staff, writer, eyes, texas,...","[amy, lupica, odp, staff, writer, eyes, texas,..."
220,44,2021-02-19 22:33:00+03:00,Daily Sabah,US cities slammed by storms face new crisis: W...,States slammed by winter storms that left mill...,States slammed by winter storms that left mill...,NaN,states slammed by winter storms that left mill...,"[states, slammed, winter, storms, left, millio...","[states, slammed, winter, storms, left, millio..."
221,0,3-Mar-21,Jackson Free Press,"PUBLISHER'S NOTE: Jackson’s Water Crisis, What...",While Lt. Gov. Delbert Hosemann often presents...,Around a long rectangle of tables in the middl...,Nick Judin Jackson water crisis,while lt gov delbert hosemann often presents h...,"[lt, gov, delbert, hosemann, often, presents, ...","[lt, gov, delbert, hosemann, often, presents, ..."
222,5,2021-03-02 01:54:57+00:00,Mississippi Free Press,Hosemann Pins Water Blame on Jackson Leaders S...,Lt. Gov. Delbert Hosemann confirmed today that...,Delbert Hosemann confirmed today that there ar...,Nick Judin Jackson water crisis,lt gov delbert hosemann confirmed today that t...,"[lt, gov, delbert, hosemann, confirmed, today,...","[lt, gov, delbert, hosemann, confirmed, today,..."


In [ ]:
#Trying for different number of topics

id2word = Dictionary(news_data['No stopword list'])

id2word.filter_extremes(no_below=2, no_above=.99)

corpus = [id2word.doc2bow(d) for d in news_data['No stopword list']]


base_model = LdaMulticore(corpus=corpus, num_topics=4, id2word=id2word, workers=12, passes=5)

words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]

# Create Topics
topics = [' '.join(t[0:10]) for t in words]

# Getting the topics
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")
    
#Visualizing topics    
pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim_models.prepare(base_model, corpus, dictionary=base_model.id2word)
vis

C:\Users\rishabhstha\AppData\Local\Programs\Python\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


------ Topic 0 ------
state infrastructure system people residents crisis mayor also million lumumba

------ Topic 1 ------
residents state still people system crisis st pressure mayor without

------ Topic 2 ------
state power residents weather county without people lumumba mayor system

------ Topic 3 ------
system winter pressure residents boil power williams people without storms



In [10]:
#With raondom state as 9
id2word = Dictionary(news_data['No stopword list'])

id2word.filter_extremes(no_below=2, no_above=.99)

corpus = [id2word.doc2bow(d) for d in news_data['No stopword list']]


base_model = LdaMulticore(corpus=corpus, num_topics=4, id2word=id2word, workers=12, passes=5, random_state=9)

words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]

# Create Topics
topics = [' '.join(t[0:10]) for t in words]

# Getting the topics
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")
    
#Visualizing topics    
pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim_models.prepare(base_model, corpus, dictionary=base_model.id2word)
vis

C:\Users\rishabhstha\AppData\Local\Programs\Python\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


------ Topic 0 ------
system residents williams pressure st road crisis still distribution mayor

------ Topic 1 ------
state million system infrastructure crisis help mayor federal would need

------ Topic 2 ------
power people state texas winter without residents weather still storm

------ Topic 3 ------
state residents lumumba people infrastructure mayor system crisis also without



PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.044567  0.072161       1        1  36.031327
3      0.006616  0.013442       2        1  25.447657
0     -0.059168 -0.073204       3        1  21.307327
1      0.097119 -0.012400       4        1  17.213689, topic_info=           Term        Freq       Total Category  logprob  loglift
349    williams  224.000000  224.000000  Default  30.0000  30.0000
1866         st  113.000000  113.000000  Default  29.0000  29.0000
200       power  366.000000  366.000000  Default  28.0000  28.0000
268      system  548.000000  548.000000  Default  27.0000  27.0000
612     million  253.000000  253.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
222   residents   61.919189  606.850371   Topic4  -5.3949  -0.5230
373       could   42.901062  206.665968   Topic4  -5.7618   0.1873
860        make   37.049351  137.557498   Topic4  -5.9084   0.4477
306     without   44.119600  385.573621   Topic4  -5.7338  -0.4084
333     lumumba   40.574752  297.144616   Topic4  -5.8175  -0.2316

[355 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
2838      1  0.906751     abbott
2838      2  0.039424     abbott
2838      4  0.039424     abbott
931       1  0.587314  according
931       2  0.191515  according
...     ...       ...        ...
3858      3  0.820597       wsba
4641      3  0.903137        wtp
4642      1  0.136698      yazoo
4642      2  0.820190      yazoo
4642      3  0.045566      yazoo

[787 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 1, 2])